<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning/blob/master/ML4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np

In [18]:
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a/sum_exp_a
  return y


In [19]:
a = np.array([0.3,2.9,4.0])
y = softmax(a)
y

array([0.01821127, 0.24519181, 0.73659691])

In [20]:
np.sum(y)
#softmax 함수의 특징. 총합은 항상 1이다. 덕분에 문제를 확률적으로 대응가능. 또한 softmax 함수는 단조증가함수. 
# 그래서 단순 분류할때는 소프트맥스 생략. 학습시킬떄는 소프트맥스 함수 사용. 
# 자연스레 분류문제에서 출력층 뉴런은 분류하고 싶은 클래스의 수만큼 설정하는것이 일반적. 


1.0

In [21]:
import sys, os
sys.path.append(os.pardir) #부모 디렉터리의 파일을 가져올 수 있도록 설정. 여기서는 사용하지않음
 

In [56]:
from mnist import load_mnist #깃허브에서 끌어왔습니다.

In [57]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
#MNIST 데이터셋을 다운받아 이미지를 넘파이 배열로 변환해주는 함수

In [58]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [59]:
from PIL import Image
#from IPython.display import Image
def img_show(img):
  pil_img = Image.fromarray(np.uint8(img))
  display(pil_img) # 대신 이거 쓰면 됌
  #pil_img.show()  #ipython에서는 이게 안됌. ex) google colabo, jupyter notebook
img= x_train[0]
label = t_train[0]
print(label)
print(img.shape)
img = img.reshape(28,28)
print(img.shape)

img_show(img) #이게 왜 안나오지
#이제 나옴

5
(784,)
(28, 28)


In [62]:
def sigmoid(x):
  return 1/ (1 + np.exp(-x))

In [64]:
import pickle
def get_data():
  (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten= True, one_hot_label = False)
  return x_test, t_test
  # x_test는 시험 이미지 배열이고, t_test는 각각에 대한 정답 배열이다
def init_network():
  with open("/content/sample_weight.pkl", 'rb') as f:
    network = pickle.load(f)
  return network
  # pickle 파일인 ..에 저장된 학습된 가중치 매개변수를 읽어와서(딕셔너리 변수로 저장되어있음) network에 저장
def predict(network, x):
  W1,W2,W3 = network['W1'], network['W2'], network['W3']
  b1,b2,b3 = network['b1'], network['b2'], network['b3']
  a1 = np.dot(x,W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1,W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2,W3) + b3
  y = softmax(a3)
  return y
  # 각 0~9 후보군 별 추정확률로 이루어진 넘파이 배열 리턴

x ,t = get_data()
network = init_network()
accuracy_cnt = 0 #그냥 맞춘 횟수
for i in range(len(x)):
  y = predict(network, x[i]) # x 의 배열 원소 하나하나 돌며 이미지 파일 각각 정답여부 확인
  p = np.argmax(y) #y중 가장 높은 확률을 차지한 인덱스 리턴. 예측결과를 리턴!~
  if p == t[i]:# 그 인덱스가 정답과 맞는가
    accuracy_cnt += 1
print("Accruacy : " + str(float(accuracy_cnt) / len(x)))
# 93,52 퍼센트. 정확도를 더 높이는 방법(hidden layer, batch...등등)은 다음에 소개
# load_mnist의 normalize을 true로 설정함. 픽셀 값 변환. 0~1의 값으로 정규화.
# 정규화 : 데이터를 특정 범위로 변환하는 처리
# 전처리 : 신경망의 입력 데이터에 특정 변환을 가하는 것
# 입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행함
# 신경망의 구조 ---------> 입력층 뉴런 : 784개, 출력층 뉴런 : 10개, 은닉층 뉴런 : 50 / 100 (이건 그냥 임의임)
# 이유 : 하나의 이미지 파일의 크기가 28 * 28 사이즈기 떄문
# 이유 : 이 문제가 0~9 에서 어떤 숫자인지 구분하는 문제이기 떄문
# 이 값들은 pkl 파일에 내장되어있음
# 이렇게 입력 데이터 배열의 인덱스 하나하나씩 predict 인공신경망 확률을 구해서 비교하는 알고리즘임
# 이때 입력 데이터를 load_mnist에서 그냥 가져오는게 아니라 normalize을 true로 켜서
# 픽셀 값을 0~1로 변환하는 (정규화하는) 과정을 전처리라고 한다
# 정리 : 입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행함

Accruacy : 0.9352


In [65]:
# 전처리는 식별 능력 개선/ 학습 속도 향상 등의 이유로 현업에서 많이 쓰임. 
# 데이터를 끌고 올때 왠만하면 좀더 가공한 것으로 가져오자는 느낌

In [68]:
#batch : 큰 거 하나가 작은거 여러개보다 컴퓨턴 훨씬 빠르게 학습
x,t = get_data()
network = init_network()
batch_size = 100
accuracy_cnt  = 0
for i in range(0, len(x), batch_size): # 간격 : batch_size 
  x_batch = x[i:i+batch_size]
  y_batch = predict(network, x_batch)
  p = np.argmax(y_batch, axis=1) # 100 *10 배열중 각각의 배열 (100 기준) [이게 1번째 차원이라는 뜻] 에서 최댓값의 인덱스 찾기. 결과값은 당연히 1*100 배열
  accuracy_cnt += np.sum(p == t[i:i+batch_size])
  # 마찬가지로 ==는 넘파이 배열끼리 간 비교한 bool 배열. true면 1로 인식. sum을 통해 결과배열에서 true가 몇개인지 세서 리턴. 결과적으로 batch 안한거랑 동일 
print("Accuracy : " + str(float(accuracy_cnt)/ len(x)))

Accuracy : 0.9352
